In [2]:
!pip install groq tavily-python python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.1 MB/s eta 0:00:00


In [4]:
import os
import json
from groq import Groq
from tavily import TavilyClient
from typing import List, Dict
import time

from google.colab import userdata


try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
except:
    # Fallback - you can directly paste your keys here (not recommended for production)
    GROQ_API_KEY = "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"
    TAVILY_API_KEY = "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"

print("Libraries imported successfully!")

Libraries imported successfully!


In [5]:
# CELL 3: Initialize API Clients
# ============================================================================

# Initialize the LLM client (Groq)
groq_client = Groq(api_key=GROQ_API_KEY)

# Initialize the web search client (Tavily)
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Test the connections
print("🧪 Testing API connections...")

try:
    # Test Groq
    test_response = groq_client.chat.completions.create(
        messages=[{"role": "user", "content": "Say 'Hello'"}],
        model="llama3-8b-8192",
        max_tokens=10
    )
    print("✅ Groq API connection successful!")

    # Test Tavily
    test_search = tavily_client.search("test", max_results=1)
    print("✅ Tavily API connection successful!")

    print("🚀 All systems ready!")

except Exception as e:
    print(f"❌ API connection failed: {e}")
    print("Please check your API keys and try again.")

🧪 Testing API connections...
✅ Groq API connection successful!
✅ Tavily API connection successful!
🚀 All systems ready!


In [6]:
class ReActAgent:
    """
    ReAct (Reasoning + Acting) Agent for Web Research

    This agent follows the ReAct pattern:
    1. PLANNING: Generate research questions using LLM
    2. ACTING: Search the web for answers
    3. SYNTHESIS: Compile results into a structured report
    """

    def __init__(self, groq_client, tavily_client):
        """Initialize the ReAct Agent with LLM and search clients"""
        self.groq_client = groq_client
        self.tavily_client = tavily_client
        self.research_data = {}
        self.topic = ""

    def generate_research_questions(self, topic: str) -> List[str]:
        """
        PLANNING PHASE: Use LLM to generate research questions

        Args:
            topic (str): The research topic

        Returns:
            List[str]: List of research questions
        """
        print(f"🧠 Generating research questions for: {topic}")

        prompt = f"""You are an expert research assistant. For the topic '{topic}', generate exactly 5-6 well-structured research questions that cover different important aspects of this topic.

Requirements for questions:
- Each question should be specific and focused
- Cover different angles/dimensions of the topic
- Be suitable for web search
- Be answerable with current information available online
- Avoid yes/no questions
- Include both factual and analytical aspects

Topic: {topic}

Generate the questions in this exact format:
1. [First question]
2. [Second question]
3. [Third question]
4. [Fourth question]
5. [Fifth question]
6. [Sixth question] (if applicable)

Only return the numbered questions, nothing else."""

        try:
            response = self.groq_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama3-8b-8192",
                temperature=0.7,
                max_tokens=600
            )

            questions_text = response.choices[0].message.content.strip()

            # Parse the numbered questions
            questions = []
            for line in questions_text.split('\n'):
                line = line.strip()
                if line and (line[0].isdigit() or line.startswith('-') or line.startswith('•')):
                    # Remove numbering and clean up
                    clean_question = line.split('.', 1)[-1].strip()
                    if clean_question:
                        questions.append(clean_question)

            print(f"✅ Generated {len(questions)} research questions:")
            for i, q in enumerate(questions, 1):
                print(f"   {i}. {q}")

            return questions

        except Exception as e:
            print(f"❌ Error generating questions: {e}")
            return [
                f"What is {topic} and why is it important?",
                f"What are the current trends in {topic}?",
                f"What are the main challenges related to {topic}?",
                f"What are the recent developments in {topic}?",
                f"What is the future outlook for {topic}?"
            ]

    def search_web(self, question: str) -> Dict:
        """
        ACTING PHASE: Search the web for answers to each question

        Args:
            question (str): The research question to search for

        Returns:
            Dict: Search results with extracted information
        """
        print(f"🔍 Searching: {question}")

        try:
            # Perform web search using Tavily
            search_results = self.tavily_client.search(
                query=question,
                search_depth="basic",
                max_results=3,
                include_answer=True,
                include_raw_content=False
            )

            # Extract relevant information
            extracted_info = {
                'question': question,
                'answer': search_results.get('answer', ''),
                'results': []
            }

            if 'results' in search_results and search_results['results']:
                for i, result in enumerate(search_results['results'][:3]):
                    content = result.get('content', 'No content available')
                    # Limit content length
                    if len(content) > 800:
                        content = content[:800] + "..."

                    extracted_info['results'].append({
                        'title': result.get('title', f'Search Result {i+1}'),
                        'url': result.get('url', ''),
                        'content': content,
                        'score': result.get('score', 0)
                    })

                print(f"   ✅ Found {len(extracted_info['results'])} relevant sources")
            else:
                print(f"   ⚠️ No results found for this question")

            return extracted_info

        except Exception as e:
            print(f"   ❌ Error searching for '{question}': {e}")
            return {
                'question': question,
                'answer': '',
                'results': []
            }

    def compile_report(self, topic: str) -> str:
        """
        SYNTHESIS PHASE: Compile all research into a structured report

        Args:
            topic (str): The research topic

        Returns:
            str: Formatted research report in markdown
        """
        print("📝 Compiling research report...")

        # Report header
        report = f"# Research Report: {topic}\n\n"

        # Add timestamp
        from datetime import datetime
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        report += f"**Generated on:** {current_time}\n\n"

        # Introduction
        report += "## Executive Summary\n\n"
        report += f"This comprehensive research report presents findings on '{topic}' gathered through systematic web research. "
        report += f"The research covered {len(self.research_data)} key areas, providing current insights and relevant information from multiple sources.\n\n"

        # Table of Contents
        report += "## Table of Contents\n\n"
        for i, question in enumerate(self.research_data.keys(), 1):
            report += f"{i}. {question}\n"
        report += "\n"

        # Main content sections
        for i, (question, data) in enumerate(self.research_data.items(), 1):
            report += f"## {i}. {question}\n\n"

            # Add direct answer if available
            if data.get('answer'):
                report += f"**Key Finding:** {data['answer']}\n\n"

            # Add detailed information from sources
            if data['results']:
                report += "### Detailed Information\n\n"
                for j, result in enumerate(data['results'], 1):
                    report += f"#### Source {j}: {result['title']}\n\n"
                    report += f"{result['content']}\n\n"
                    if result['url']:
                        report += f"**Source:** [{result['url']}]({result['url']})\n\n"
                    report += "---\n\n"
            else:
                report += "*No specific information found for this question through web search.*\n\n"

        # Conclusion
        report += "## Conclusion\n\n"
        report += f"This research on '{topic}' has provided comprehensive insights across {len(self.research_data)} key areas. "
        report += "The findings represent current information gathered from authoritative web sources and can serve as a foundation "
        report += "for further research, decision-making, or academic purposes.\n\n"

        # Sources summary
        total_sources = sum(len(data['results']) for data in self.research_data.values())
        report += f"**Total Sources Consulted:** {total_sources}\n"
        report += f"**Research Questions Covered:** {len(self.research_data)}\n\n"

        report += "---\n"
        report += "*This report was generated using automated web research and may require fact-checking for critical applications.*\n"

        return report

    def research(self, topic: str) -> str:
        """
        Main method that orchestrates the entire ReAct process

        Args:
            topic (str): The topic to research

        Returns:
            str: Complete research report
        """
        self.topic = topic
        self.research_data = {}  # Reset previous research

        print(f"🚀 Starting ReAct research process for: '{topic}'")
        print("=" * 60)

        # Step 1: PLANNING - Generate research questions
        print("\n📋 PHASE 1: PLANNING - Generating research questions...")
        questions = self.generate_research_questions(topic)

        if not questions:
            return "❌ Failed to generate research questions. Please try again."

        # Step 2: ACTING - Search for each question
        print(f"\n🔍 PHASE 2: ACTING - Searching for answers to {len(questions)} questions...")
        for i, question in enumerate(questions, 1):
            print(f"\n[{i}/{len(questions)}] Processing question...")
            search_data = self.search_web(question)
            self.research_data[question] = search_data

            # Be respectful to APIs - small delay between requests
            if i < len(questions):
                time.sleep(2)

        # Step 3: SYNTHESIS - Compile the report
        print(f"\n📊 PHASE 3: SYNTHESIS - Compiling comprehensive report...")
        report = self.compile_report(topic)

        print(f"\n✅ Research completed successfully!")
        print(f"📈 Generated report covering {len(self.research_data)} research areas")
        print(f"📄 Report length: {len(report.split())} words")

        return report

print("🎯 ReAct Agent class defined successfully!")
print("📋 Features included:")
print("   • Intelligent question generation")
print("   • Web search with multiple sources")
print("   • Structured report compilation")
print("   • Error handling and recovery")
print("   • Progress tracking")

🎯 ReAct Agent class defined successfully!
📋 Features included:
   • Intelligent question generation
   • Web search with multiple sources
   • Structured report compilation
   • Error handling and recovery
   • Progress tracking


In [7]:
agent = ReActAgent(groq_client, tavily_client)
print("🤖 ReAct Agent instance created!")

# Test with a research topic
print("\n" + "="*50)
print("🧪 TESTING THE AGENT")
print("="*50)

# You can change this topic to anything you want to research
research_topic = "Artificial Intelligence in Healthcare 2024"

print(f"🎯 Research Topic: {research_topic}")
print("\nStarting research process...\n")

# Run the research (this will take a few minutes)
final_report = agent.research(research_topic)

🤖 ReAct Agent instance created!

🧪 TESTING THE AGENT
🎯 Research Topic: Artificial Intelligence in Healthcare 2024

Starting research process...

🚀 Starting ReAct research process for: 'Artificial Intelligence in Healthcare 2024'

📋 PHASE 1: PLANNING - Generating research questions...
🧠 Generating research questions for: Artificial Intelligence in Healthcare 2024
✅ Generated 6 research questions:
   1. What are the most widely adopted AI-powered healthcare applications in 2024, and what are their primary benefits in terms of patient outcomes and healthcare resource utilization?
   2. How have advancements in natural language processing and machine learning enabled AI systems to improve diagnosis accuracy and patient engagement in healthcare, particularly in high-risk patient populations?
   3. What are the key regulatory and ethical challenges associated with the integration of AI in healthcare, and how are policymakers and healthcare organizations addressing these concerns in 2024?
   

In [8]:
print("\n" + "="*80)
print("📋 FINAL RESEARCH REPORT")
print("="*80)
print()
print(final_report)


📋 FINAL RESEARCH REPORT

# Research Report: Artificial Intelligence in Healthcare 2024

**Generated on:** 2025-06-13 04:07:51

## Executive Summary

This comprehensive research report presents findings on 'Artificial Intelligence in Healthcare 2024' gathered through systematic web research. The research covered 6 key areas, providing current insights and relevant information from multiple sources.

## Table of Contents

1. What are the most widely adopted AI-powered healthcare applications in 2024, and what are their primary benefits in terms of patient outcomes and healthcare resource utilization?
2. How have advancements in natural language processing and machine learning enabled AI systems to improve diagnosis accuracy and patient engagement in healthcare, particularly in high-risk patient populations?
3. What are the key regulatory and ethical challenges associated with the integration of AI in healthcare, and how are policymakers and healthcare organizations addressing these conc

In [9]:
import re

def create_filename(topic):
    # Clean the topic for filename
    clean_topic = re.sub(r'[^\w\s-]', '', topic)
    clean_topic = re.sub(r'[\s]+', '_', clean_topic)
    return f"research_report_{clean_topic.lower()}.md"

filename = create_filename(research_topic)

# Save the report
try:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(final_report)
    print(f"💾 Report saved as: {filename}")

    # Download the file in Colab
    from google.colab import files
    files.download(filename)
    print("📥 File download started!")

except Exception as e:
    print(f"❌ Error saving file: {e}")

💾 Report saved as: research_report_artificial_intelligence_in_healthcare_2024.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 File download started!


In [11]:
# ============================================================================
# ENHANCED INTERACTIVE REACT AGENT - FIXED VERSION
# Add this cell to your existing notebook
# ============================================================================

def interactive_research_agent():
    """
    Interactive function that allows users to input research topics
    and get comprehensive reports
    """
    print("🤖 Welcome to the Interactive ReAct Research Agent!")
    print("=" * 60)
    print("This AI agent can research any topic and generate professional reports.")
    print("Just enter a topic and I'll handle the rest!")
    print()

    while True:
        print("\n📋 MENU OPTIONS:")
        print("1. 🔍 Research a new topic")
        print("2. 📊 Quick topic analysis (faster)")
        print("3. 🎯 Ask a specific question")
        print("4. 📚 View research history")
        print("5. 🚪 Exit")

        try:
            choice = input("\n➤ Enter your choice (1-5): ").strip()

            if choice == "1":
                # Full research mode
                topic = input("\n📝 Enter the topic you want to research: ").strip()

                if not topic:
                    print("❌ Please enter a valid topic.")
                    continue

                print(f"\n🚀 Starting comprehensive research on: '{topic}'")
                print("⏳ This may take 2-3 minutes...")

                # Create new agent instance for this research
                research_agent = ReActAgent(groq_client, tavily_client)

                try:
                    # Perform full research
                    report = research_agent.research(topic)

                    # Display summary
                    word_count = len(report.split())
                    question_count = len(research_agent.research_data)

                    print(f"\n✅ Research completed!")
                    print(f"📊 Report stats: {word_count} words, {question_count} research questions")

                    # Ask if user wants to see the full report
                    show_report = input("\n🔍 Show full report? (y/n): ").strip().lower()
                    if show_report in ['y', 'yes']:
                        print("\n" + "="*60)
                        print("FULL RESEARCH REPORT")
                        print("="*60)
                        print(report)

                    # Save option
                    save_option = input("\n💾 Save report to file? (y/n): ").strip().lower()
                    if save_option in ['y', 'yes']:
                        filename = create_filename(topic)
                        with open(filename, 'w', encoding='utf-8') as f:
                            f.write(report)
                        print(f"✅ Report saved as: {filename}")

                        # Download option
                        download_option = input("📥 Download file? (y/n): ").strip().lower()
                        if download_option in ['y', 'yes']:
                            from google.colab import files
                            files.download(filename)
                            print("📥 Download started!")

                except Exception as e:
                    print(f"❌ Error during research: {e}")

            elif choice == "2":
                # Quick analysis mode
                topic = input("\n📝 Enter topic for quick analysis: ").strip()

                if not topic:
                    print("❌ Please enter a valid topic.")
                    continue

                print(f"\n⚡ Quick analysis of: '{topic}'")

                try:
                    # Generate fewer questions for faster analysis
                    quick_agent = ReActAgent(groq_client, tavily_client)

                    # Override the question generation for quick mode
                    quick_questions = [
                        f"What is {topic} and why is it important?",
                        f"What are the current trends in {topic}?",
                        f"What are the key benefits and challenges of {topic}?"
                    ]

                    print(f"🔍 Researching 3 key aspects...")

                    for i, question in enumerate(quick_questions, 1):
                        print(f"[{i}/3] {question}")
                        search_data = quick_agent.search_web(question)
                        quick_agent.research_data[question] = search_data
                        time.sleep(1)

                    # Generate quick summary
                    quick_report = quick_agent.compile_report(topic)

                    print(f"\n✅ Quick analysis completed!")
                    print("\n📋 QUICK ANALYSIS SUMMARY:")
                    print("="*40)

                    # Show just the key findings
                    for question, data in quick_agent.research_data.items():
                        print(f"\n❓ {question}")
                        if data.get('answer'):
                            print(f"💡 {data['answer']}")
                        elif data['results']:
                            print(f"💡 {data['results'][0]['content'][:200]}...")
                        else:
                            print("💡 No specific information found")

                except Exception as e:
                    print(f"❌ Error during quick analysis: {e}")

            elif choice == "3":
                # Specific question mode
                question = input("\n❓ Ask me any specific question: ").strip()

                if not question:
                    print("❌ Please enter a valid question.")
                    continue

                print(f"\n🔍 Searching for: '{question}'")

                try:
                    # Direct search for specific question
                    question_agent = ReActAgent(groq_client, tavily_client)
                    search_result = question_agent.search_web(question)

                    print(f"\n💡 ANSWER:")
                    print("="*30)

                    if search_result.get('answer'):
                        print(f"✅ {search_result['answer']}")

                    if search_result['results']:
                        print(f"\n📚 SOURCES:")
                        for i, result in enumerate(search_result['results'][:2], 1):
                            print(f"\n{i}. {result['title']}")
                            print(f"   {result['content'][:300]}...")
                            if result['url']:
                                print(f"   🔗 {result['url']}")
                    else:
                        print("❌ No specific sources found for this question.")

                except Exception as e:
                    print(f"❌ Error searching for question: {e}")

            elif choice == "4":
                # Show research history (if we implement session storage)
                print("\n📚 RESEARCH HISTORY:")
                print("This feature would show your previous research topics.")
                print("(Currently not implemented - would require session storage)")

            elif choice == "5":
                print("\n👋 Thank you for using the ReAct Research Agent!")
                print("🎯 Remember: You can research any topic, anytime!")
                break

            else:
                print("❌ Invalid choice. Please enter 1, 2, 3, 4, or 5.")

        except KeyboardInterrupt:
            print("\n\n👋 Research interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            print("Please try again.")

# ============================================================================
# SIMPLE QUESTION-ANSWER FUNCTION
# ============================================================================

def ask_agent(question):
    """
    Simple function to ask the agent a single question

    Args:
        question (str): The question to ask

    Returns:
        dict: Answer and sources
    """
    try:
        agent = ReActAgent(groq_client, tavily_client)
        result = agent.search_web(question)

        response = {
            'question': question,
            'answer': result.get('answer', 'No direct answer found'),
            'sources': []
        }

        for source in result['results']:
            response['sources'].append({
                'title': source['title'],
                'content': source['content'][:200] + '...' if len(source['content']) > 200 else source['content'],
                'url': source['url']
            })

        return response

    except Exception as e:
        return {
            'question': question,
            'answer': f'Error: {str(e)}',
            'sources': []
        }

# ============================================================================
# BATCH QUESTION PROCESSOR
# ============================================================================

def process_multiple_questions(questions_list):
    """
    Process multiple questions at once

    Args:
        questions_list (list): List of questions to process

    Returns:
        dict: Results for all questions
    """
    results = {}
    agent = ReActAgent(groq_client, tavily_client)

    print(f"🔄 Processing {len(questions_list)} questions...")

    for i, question in enumerate(questions_list, 1):
        print(f"[{i}/{len(questions_list)}] {question}")

        try:
            result = agent.search_web(question)
            results[question] = {
                'answer': result.get('answer', 'No answer found'),
                'sources_count': len(result['results']),
                'top_source': result['results'][0] if result['results'] else None
            }

            # Small delay between questions
            if i < len(questions_list):
                time.sleep(1)

        except Exception as e:
            results[question] = {
                'answer': f'Error: {str(e)}',
                'sources_count': 0,
                'top_source': None
            }

    return results

# ============================================================================
# QUICK TEST FUNCTIONS
# ============================================================================

def quick_test():
    """Quick test function to verify everything works"""
    print("🧪 Running quick test...")

    try:
        # Test single question
        response = ask_agent("What is artificial intelligence?")
        print("✅ Single question test passed!")
        print(f"Answer preview: {response['answer'][:100]}...")

        # Test multiple questions
        test_questions = [
            "What is machine learning?",
            "What are the benefits of renewable energy?"
        ]
        results = process_multiple_questions(test_questions)
        print("✅ Multiple questions test passed!")

        print("\n🎉 All tests passed! Your interactive agent is ready!")

    except Exception as e:
        print(f"❌ Test failed: {e}")
        print("Please check your API keys and connections.")

# ============================================================================
# READY TO USE!
# ============================================================================

print("🎉 Interactive features added successfully!")
print("\n🚀 Available functions:")
print("   • interactive_research_agent() - Full interactive mode")
print("   • ask_agent('your question') - Quick single question")
print("   • process_multiple_questions([list]) - Batch processing")
print("   • quick_test() - Test if everything works")
print("\n💡 Try running: interactive_research_agent()")



🎉 Interactive features added successfully!

🚀 Available functions:
   • interactive_research_agent() - Full interactive mode
   • ask_agent('your question') - Quick single question
   • process_multiple_questions([list]) - Batch processing
   • quick_test() - Test if everything works

💡 Try running: interactive_research_agent()


In [12]:
interactive_research_agent()

🤖 Welcome to the Interactive ReAct Research Agent!
This AI agent can research any topic and generate professional reports.
Just enter a topic and I'll handle the rest!


📋 MENU OPTIONS:
1. 🔍 Research a new topic
2. 📊 Quick topic analysis (faster)
3. 🎯 Ask a specific question
4. 📚 View research history
5. 🚪 Exit

➤ Enter your choice (1-5): 1

📝 Enter the topic you want to research: pineapple

🚀 Starting comprehensive research on: 'pineapple'
⏳ This may take 2-3 minutes...
🚀 Starting ReAct research process for: 'pineapple'

📋 PHASE 1: PLANNING - Generating research questions...
🧠 Generating research questions for: pineapple
✅ Generated 6 research questions:
   1. What are the top pineapple-producing countries in the world and what are the primary factors contributing to their high production levels?
   2. How does the nutritional profile of pineapple vary across different varieties, and what are the implications for human health and nutrition?
   3. What are the most effective methods fo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download started!

📋 MENU OPTIONS:
1. 🔍 Research a new topic
2. 📊 Quick topic analysis (faster)
3. 🎯 Ask a specific question
4. 📚 View research history
5. 🚪 Exit

➤ Enter your choice (1-5): 5

👋 Thank you for using the ReAct Research Agent!
🎯 Remember: You can research any topic, anytime!
